In [6]:
pip install rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [53]:
# api_chatbot/faq.py

import pandas as pd
from rapidfuzz import process, fuzz

# 1. Charger les données FAQ (en anglais)
faq_df = pd.read_csv("/Users/ines/NLP/emergency_chatbot/data/medquad_parsed.csv")

# 2. Supprimer les doublons éventuels basés sur la question
faq_df = faq_df.drop_duplicates(subset=["question"])

# 3. Nettoyer les questions (retirer les espaces inutiles, mettre en minuscules)
faq_df['question'] = faq_df['question'].astype(str).str.strip().str.lower()

# 4. Créer un dictionnaire question -> réponse
FAQ_DB = dict(zip(faq_df['question'], faq_df['answer']))

# Mini dictionnaire FAQ basique pour site urgences (en anglais)
FAQ_DB_BASIC = {
    "what are your opening hours?": "We are open 7 days a week, from 8 AM to 8 PM.",
    "what is the address of the hospital?": "Our address is 123 Health Street.",
    "what is the emergency phone number?": "Dial 911 for any life-threatening emergency.",
    "do you accept walk-in patients?": "Yes, we accept walk-in patients during opening hours.",
    "how can i make an appointment?": "You can make an appointment by calling our reception or through our website.",
    "are emergency services available 24/7?": "Yes, our emergency services operate 24/7.",
    "is parking available at the hospital?": "Yes, parking is available for patients and visitors.",
    "do you have a pharmacy on site?": "Yes, there is a pharmacy inside the hospital building.",
    "can i get a covid-19 test here?": "Yes, COVID-19 testing is available by appointment.",
    "what insurance plans do you accept?": "We accept most major insurance plans. Please contact billing for details."
}

FAQ_DB = {
    **FAQ_DB,          # ton dictionnaire actuel (ex: celui venant du CSV)
    **FAQ_DB_BASIC     # le mini dictionnaire basique d'urgence
}

# 5. Fonction pour faire du fuzzy matching sur une question utilisateur
def find_faq_answer(user_input, threshold=69):
    """
    Retourne la meilleure réponse correspondante dans la FAQ si le score est suffisant.

    Paramètres :
        user_input (str) : la question posée par l'utilisateur
        threshold (int) : le score minimal pour accepter une correspondance

    Retour :
        str ou None : la réponse correspondante ou None si rien de pertinent trouvé
    """
    questions_list = list(FAQ_DB.keys())
    cleaned_input = user_input.strip().lower()

    match, score, _ = process.extractOne(
        cleaned_input, questions_list, scorer=fuzz.token_sort_ratio
    )

    if score >= threshold:
        return FAQ_DB[match]
    else:
        return None

display(faq_df.head())

# 6. Exemple d'utilisation
if __name__ == "__main__":
    user_question = "What is keratoderma?"
    answer = find_faq_answer(user_question)

    if answer:
        print(f"Réponse FAQ : {answer}")
    else:
        print("Sorry, I couldn't find a relevant answer in the FAQ.")


,question,answer,domain,doc_id,dial_id,references
0,what is (are) keratoderma with woolly hair ?,Keratoderma with woolly hair is a group of rel...,GHR,0000559,NaN,['https://ghr.nlm.nih.gov/condition/keratoderm...
1,how many people are affected by keratoderma wi...,Keratoderma with woolly hair is rare; its prev...,GHR,0000559,NaN,['https://ghr.nlm.nih.gov/condition/keratoderm...
2,what are the genetic changes related to kerato...,"Mutations in the JUP, DSP, DSC2, and KANK2 gen...",GHR,0000559,NaN,['https://ghr.nlm.nih.gov/condition/keratoderm...
3,is keratoderma with woolly hair inherited ?,Most cases of keratoderma with woolly hair hav...,GHR,0000559,NaN,['https://ghr.nlm.nih.gov/condition/keratoderm...
4,what are the treatments for keratoderma with w...,These resources address the diagnosis or manag...,GHR,0000559,NaN,['https://ghr.nlm.nih.gov/condition/keratoderm...


Sorry, I couldn't find a relevant answer in the FAQ.
